In this notebook, we run an optimization making use of all of the functionality of horsetail matching. 

Things to note...

When specifying undertainties as sampling functions, the number of samples returned should be equal to that provided in the samples_prob argument.

When evaluating the metric, only integration points that are close to the range of samples obtained are evaluated, meaning a very large number and range of integration points (which should cover the range of values of the qoi that might be seen) can be provided without excessive calculation being performed. 

When using the built in Polynomial Chaos surrogate, the quadrature points used to evaluate the surrogate must match those given in the surrogate_points argument. 

The uncertainties are stored internally with probabilistic uncertainties first, followed by interval uncertainties. Therefore when using surrogate models, this order should match that used by the surrogate.

In [4]:
import numpy as np
import scipy.optimize as scopt

from horsetailmatching import HorsetailMatching, UniformParameter, GaussianParameter, IntervalParameter, UncertainParameter
from horsetailmatching.demoproblems import TP2
from horsetailmatching.surrogates import PolySurrogate

def fQOI(x, u):
    factor = np.sqrt(u[0]**2 + u[1]**2 + u[2]**2 + 1)
    q = 1 + factor*(x[0]**2 + x[1]*x[0] + x[1]**2)
    grad = [factor*(2*x[0] + x[1]), factor*(x[0] + 2*x[1])]
    return q, grad

def myPDF(q):
    if q > 1 or q < -1:
        return 0
    else:
        return 0.5
    
n_samples_prob = 100
n_samples_int = 10

u_1 = UncertainParameter(myPDF, lower_bound=-1, upper_bound=1)
u_2 = lambda : np.random.normal(0, 1, n_samples_prob)
u_3 = IntervalParameter(lower_bound=-1, upper_bound=1)
             
def ftarget_u(h):
    return 0 - h**5

def ftarget_l(h):
    return 1 - h**5
             
qPolyChaos = PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])
gradPolyChaos = [PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre']),
                 PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])]
u_quad_points = qPolyChaos.getQuadraturePoints()
             
def mySurrogateWithGrad(u_quad, q_quad, grad_quad):
    qPolyChaos.train(q_quad)
    for i, gPC in enumerate(gradPolyChaos):
        gPC.train(grad_quad[:, i])
    def qmodel(u):
        return qPolyChaos.predict(u)
    def gradmodel(u):
        return [gPC.predict(u) for gPC in gradPolyChaos]
    return qmodel, gradmodel

theHM = HorsetailMatching(fQOI, prob_uncertainties=[u_1, u_2], int_uncertainties=u_3, 
                          jac=True, method='kernel', ftarget=(ftarget_u, ftarget_l),
                          samples_prob=n_samples_prob, samples_int=n_samples_int, 
                          integration_points=np.linspace(-50, 1000, 2000),
                          surrogate=mySurrogateWithGrad, surrogate_points=u_quad_points, surrogate_jac=True,
                          verbose=True)

solution = scopt.minimize(theHM.evalMetric, x0=[3, 2], bounds=[(-5, 5), (-5, 5)], jac=True, method='SLSQP')

----------
At design: [3. 2.]
Evaluating surrogate
Getting uncertain parameter samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 50.0306015529
Gradient: [20.635361177419956, 18.055941030242472]
----------
At design: [-5. -5.]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 194.563105867
Gradient: [-38.72031491206719, -38.72031491206666]
----------
At design: [ 0.32596505 -0.33978058]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 2.52216836791
Gradient: [0.3648343133522796, -0.413276306213772]
----------
At design: [-0.04565756  0.06762066]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 2.40785109661
Gradient: [-0.02308350354527805, 0.08727390546024198]
----------
At design: [-0.00216825  0.01315345]
Evaluating surrogate
Re-using stored samples
Evaluating quantit

For other tutorials, see: http://www-edc.eng.cam.ac.uk/aerotools/horsetailmatching/